# Ejemplo: Estrategias Evolutivas (EE)

## Función de Ackley

La función de Ackley es una función de $n$ variables y está definida como sigue:

\begin{equation}
  \min f(\vec{x}) = -20\exp \left( -0.2 \sqrt{\frac{1}{n} \sum_{i=1}^n x_i^2} \right) 
  - exp \left( \frac{1}{n} \sum_{i=1}^n \cos (2\pi x_i) \right)
  + 20 + e
\end{equation}

donde $30 \leq x_i \leq 30$. El mínimo global está en $x_i = 0 $ y $f(\vec{x})=0$.

## Principales componentes de EE

Librerías de **Python** que vamos a utilizar.

In [3]:
import numpy as np
from copy import copy
from math import e, exp, sqrt, pi, cos
from random import choices

Datos de entrada.

In [4]:
n = 2 #Número de variables de decisión

### Representación de un individuo

Cada individuo será visto como un arreglo de valores flotantes de tamaño $n+1$. Las primeras $n$ componentes corresponden a las variables de decisión del problema y la siguente componente corresponde al tamaño de paso para la mutación de las variables del problema.

In [5]:
#Si n = 2
i = np.array([-10.3, 7.84, 0.84])
i

array([-10.3 ,   7.84,   0.84])

### Aptitud

La aptitud de un individuo $i$ la definimos como $F_a(i) = -f(x^{(i)}_1, x^{(i)}_2)$, donde $f$ es la función de Ackley.

In [7]:
def f(x):
    part1 = 20*exp(-0.2*sqrt(np.average(x**2)))
    part2 = exp(np.average(np.array([cos(i) for i in  2*pi*x])))
    return -part1 - part2 + 20 + e

def fitness(x):
    return -f(x)

In [8]:
print ("x = (%s, %s)\t-f(x) = %s\t"%(i[0], i[1], fitness(i[:2])))

x = (-10.3, 7.84)	-f(x) = -18.39186319125107	


### Población inicial

Por cada individuo requerido, se utiliza una distribución uniforme para generar números aleatorios. Primero se generan $n$ números en el intervalo $[-30, 30]$, cada uno corresponde a una variable de decisión. Posteriormente, un número aleatorio en el intervalo $(0,1)$ que corresponde al tamaño de paso.

In [15]:
def getInitialPopulation(mu, n):
    population = []
    for i in range(mu):
        #Generamos un individuo
        p = np.concatenate((np.random.uniform(-30, 30, n), np.random.uniform(0, 1, 1)))
        #Calculamos la aptitud del individuo
        p = [p, fitness(p[:n])] 
        #Agregamos al individuo a la población "parents"
        population.append(p)
        
    return population

In [16]:
mu = 100 #Tamaño de la población
population = getInitialPopulation(mu, n)
population[0]

[array([-18.65372343,  -9.19726163,   0.87925505]), -20.776871049837613]

### Cruza

#### Recombinación discreta local

Utilizando una distribución uniforme, generamos un número aleatorio en el intervalo $(0, 1)$ por cada componente de nuestro individuo. Recordemos que las primeras $n$ componentes  del individuo corresponden a sus variables de decisión y la última componente a su tamaño de paso. Si el valor es menor a 0.5, el hijo se queda con el valor del primer padre. De lo contrario, el hijo se queda con el valor del segundo padre.

In [18]:
def localDiscreteCrossover(parent_1, parent_2):
    N = len(parent_1)
    #Creamos un vector con N componentes en el intervalo
    # (0,1) con distribución uniforme
    z = np.random.uniform(0, 1, N)
    child = np.array([0.0]*N)
    #Asignamos el valor del padre correspondiente
    for i in range(N):
        if z[i] < 0.5:
            child[i] = parent_1[i]
        else:
            child[i] = parent_2[i]
            
    #Regresamos el individuo generado
    return child

In [21]:
i1 = [-10.3, 7.84, 0.84]
i2 = [2.4, -3.84, 0.98]
new_i = localDiscreteCrossover(i1,i2)
print(new_i)

[-10.3   -3.84   0.84]


### Mutación

Para mutar un individuo, se necesita mutar primero el tamaño de paso:

  \begin{eqnarray*}
    {\sigma}' = \sigma \cdot e^{ \tau \cdot N(0,1)}
  \end{eqnarray*}

Si el valor de ${\sigma}'$ es menor a $\varepsilon_0$, hacemos que ${\sigma}' = \varepsilon_0$. Posteriormente, utilizamos ${\sigma}'$ para mutar las variables de decisión. Si $n=2$, nuestras variables de decisión mutadas quedan de la siguiente forma:

\begin{eqnarray*}
    x'_1 = x_1 + \sigma' \cdot N(0,1) \\
    x'_2 = x_2 + \sigma' \cdot N(0,1)
\end{eqnarray*}

Revisamos que las variables de decisión generadas estén dentro del rango que indica el problema. En caso de que no, se les asigna el valor del límite que rebasó. Finalmente, construimos el individuo hijo:

\begin{equation*}
    i' = [x'_1, x'_2, \sigma']
\end{equation*}

In [23]:
def mutation(i, n, epsilon, tau):
    #Mutamos el tamaño de paso
    mutation_sigma = i[-1]*(e**(tau*np.random.normal(0, 1, 1)))
    #Verificamos que el nuevo valor no sea menor a épsilon
    if mutation_sigma[0] < epsilon:
        mutation_sigma[0] = epsilon

    #Mutamos las variables de decisión
    mutation_x = i[:n] + (mutation_sigma[0]*np.random.normal(0, 1, n))

    #Revisamos que estén dentro de los límites
    mutation_x[mutation_x < -30] = -30
    mutation_x[mutation_x >  30] =  30
    
    return np.concatenate((mutation_x, mutation_sigma))

In [24]:
tau = 1/sqrt(2) #Tasa de aprendizaje
epsilon = 0.01 #mínimo valor de sigma
i = [-10.3, 7.84, 0.84]
new_i = mutation(i, n, epsilon, tau)
print(new_i)

[-10.15890813   7.69605627   0.4825125 ]


#### Crear hijo usando cruza y mutación

In [25]:
def createChild(parent_1, parent_2, n, epsilon, tau):
    new_ind = localDiscreteCrossover(parent_1, parent_2)
    new_ind = mutation(new_ind, n, epsilon, tau)

    #Regresamos el nuevo individuo, junto con su valor de apitud
    return [new_ind, fitness(new_ind[:n])]

In [26]:
createChild(population[0][0], population[1][0], n, epsilon, tau)

[array([  9.79058706, -19.6921706 ,   1.24097226]), -20.876622230156865]

## Algoritmo completo de $(\mu, \lambda)$ - EE 

Se usa una selección $(\mu,\lambda)$ donde $\mu$=100 y $\lambda$=700.
Se elegirán dos padres al azar para generar un hijo nuevo y este proceso se llevará a cabo 700 veces. De los hijos resultantes, se elegirán a los 100 mejores que pasarán a la siguiente generación.

In [27]:
def EvolutionStrategies(n, G, epsilon=0.01, tau=None, mu_=100, lambda_=700):
    #Si el usuario no define tau, 
    #se usa el valor recomendado en la literatura
    if tau == None:
        tau = 1/sqrt(n)
        
    population = getInitialPopulation(mu_, n)
    #Generamos los índices de los individuos para poder elegirlos
    #aleatoriamente
    population_idx = range(mu_)
    
    #Repetimos el proceso por G generaciones
    for _ in range(G):
        offspring = []
        #Generamos lambda_ hijos
        for i in range(lambda_):
            #Seleccionamos a los padres
            parents_idx = choices(population_idx, k=2)
            parent_1 = population[population_idx[0]][0]
            parent_2 = population[population_idx[1]][0]
            #Creamos el hijo y lo añadimos a la población de hijos
            offspring.append( createChild(parent_1, parent_2, n, epsilon, tau) )

        #Ordenamos del mejor individuo al peor individuo
        offspring.sort(key=lambda x: x[-1], reverse=True)
        #Seleccionamos a los mu_ mejores
        population = offspring[:mu_].copy()
    
    
    #Regresamos el mejor individuo
    return population[0][0], -population[0][1]

In [28]:
n = 2 #Número de variables/dimensiones
mu_ = 100 #tamaño de la población
lambda_ = 700 #Cantidad de hijos que se generan por generación
G = 100 #número de generaciones
tau = 1/sqrt(n) #tasa de aprendizaje
epsilon = 0.01 #valor mínimo de sigma (tamaño de paso)

In [29]:
sol, fx = EvolutionStrategies(n, G, epsilon, tau, mu_, lambda_)
print(f"x: {sol[:n]}\nSigma: {sol[n:]}\nf(x): {fx}")

x: [-0.00065657  0.00037216]
Sigma: [0.01]
f(x): 0.002149800797706991


Realizamos $m$ ejecuciones de nuestro algoritmo usando 5, 10 y 20 variables de decisión. Para cada instancia del problema, obtenemos la corrida que encontró la mejor solución, la corrida que encontró la peor solución, la corrida que se encuentra en la mediana de las $m$ ejecuciones, la media y la desviación estándar de las $m$ corridas con respecto al valor de la función objetivo.

In [30]:
def makeMRuns(m, n, G, epsilon, tau):
    sol =[]
    for m in range(m):
        x, fx = EvolutionStrategies(n, G, epsilon, tau)
        sol.append([x, fx])

    print(f"Para {n} variables")

    sol.sort(key=lambda individuo: individuo[-1])
    print("*************** Mejor solución ***************")
    print(f"x: {sol[0][0][:n]}\nSigma: {sol[0][0][n:]}\nf(x): {sol[0][-1]}")

    print("\n*************** Peor solución ****************")
    print(f"x: {sol[-1][0][:n]}\nSigma: {sol[-1][0][n:]}\nf(x): {sol[-1][-1]}")

    print("\n****************** Mediana *******************")
    print(f"x: {sol[m//2][0][:n]}\nSigma: {sol[m//2][0][n:]}\nf(x): {sol[m//2][-1]}")

    f_sol = [x[-1] for x in sol]
    print("\nMedia: ", np.mean(f_sol))
    print("\nDesviación estándar: ", np.std(f_sol))

In [31]:
#Instancia con 5 variables
n = 5 
G = 100 
epsilon = 0.01
tau = 1/sqrt(n)
m = 21

makeMRuns(m, n, G, epsilon, tau)

Para 5 variables
*************** Mejor solución ***************
x: [ 0.00064673 -0.00386858  0.00048901  0.00019717 -0.00088428]
Sigma: [0.01]
f(x): 0.007429178001150394

*************** Peor solución ****************
x: [ 29.99756535 -24.9972199  -14.9996601  -13.9957628   26.00348506]
Sigma: [0.01047811]
f(x): 19.79532217626284

****************** Mediana *******************
x: [ 1.29892061e+01  2.99840510e+00 -9.99321299e+00  3.37022526e-04
 -8.98909928e+00]
Sigma: [0.01]
f(x): 16.324552661876236

Media:  9.776078373439958

Desviación estándar:  9.336075319808344


In [32]:
#Instancia con 10 variables
n = 10
G = 100
epsilon = 0.01
tau = 1/sqrt(n)
m = 21

makeMRuns(m, n, G, epsilon, tau)

Para 10 variables
*************** Mejor solución ***************
x: [-0.00355678 -0.00469178  0.00141009  0.00297665  0.00214689 -0.0053407
 -0.00626911 -0.0019492   0.00068978 -0.00434212]
Sigma: [0.01262651]
f(x): 0.015775159267399363

*************** Peor solución ****************
x: [ 28.99420216  28.99560935   8.99479738 -25.99707964 -19.00635308
  21.00779196  27.99865091   5.00792837  -5.99960455  -1.00384787]
Sigma: [0.01]
f(x): 19.650596767560053

****************** Mediana *******************
x: [ 3.00131095e+00 -1.29979440e+01 -9.99735252e+00 -1.50097207e-03
  1.09964389e+01  3.00359373e+00  3.99191258e+00  8.99595040e+00
  1.99896535e+01  1.29886174e+01]
Sigma: [0.01]
f(x): 17.482527440612373

Media:  12.858952654109073

Desviación estándar:  8.197688889476673


In [33]:
#Instancia con 20 variables
n = 20 
G = 100 
epsilon = 0.015 
tau = 1/sqrt(n)
m = 21

makeMRuns(m, n, G, epsilon, tau)

Para 20 variables
*************** Mejor solución ***************
x: [-0.00096883 -0.01002416 -0.01067195 -0.00834144  0.00048528  0.01650114
  0.00474224  0.0038431   0.00660964 -0.00306943  0.01213031 -0.0023433
  0.00077898 -0.01847668 -0.00778009 -0.00873538  0.00346705  0.00676743
 -0.01373322  0.01753902]
Sigma: [0.015]
f(x): 0.043170523843248265

*************** Peor solución ****************
x: [ 21.00506099  12.97651264  12.99219435  15.00800424 -10.00504334
 -15.99002891  -0.99609665  21.01679554   4.00619211   3.99273967
  21.00221368 -22.99073379  15.00043309 -22.99548788  20.98875876
 -12.00360914  18.00666284 -13.98518125   8.01997577 -16.98441983]
Sigma: [0.01602363]
f(x): 19.16371926140832

****************** Mediana *******************
x: [-1.49840187e+01  5.99552206e+00 -6.01179188e+00 -2.49974553e+01
 -1.00433095e+00  7.72362131e-03  4.00673058e+00 -1.50045372e+01
 -8.98240037e+00  9.77063376e-01  5.00800423e+00 -1.09996444e+01
  9.01293109e+00 -7.99212644e+00 -6.9901